In [1]:
pip install pytelegrambotapi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 210 kB 14.2 MB/s 
  Created wheel for pytelegrambotapi: filename=pyTelegramBotAPI-4.7.0-py3-none-any.whl size=192826 sha256=2f97af81454716bfa9463135ce54e2058c8f513d43bd10fbb76102f5e083e12e
  Stored in directory: /root/.cache/pip/wheels/ba/13/da/8abf941f7cf9f993cde6118a56a7c24e12ed791507acd8ea39
Successfully built pytelegrambotapi


In [14]:
import telebot as tb
from telebot import types
import requests as rq
import random as rd


# Конечная точка сайта с голосами птиц для России
obj_url = "https://www.xeno-canto.org/api/2/recordings?query=cnt:Russian"
wikiEndPoint = 'https://ru.wikipedia.org/api/rest_v1/page/'
username = 'koral5819@yandex.ru'
password = 'D#-R723wLcV'

bot_token = "5621803997:AAGsNlOxN6rfxkFS_18TzcnTpmf6LWAa0mc" # токен, полученный при создании бота

objPhotoUrl = ''
objDescr = ''

class Obj:
  def __init__(self, obj_spec, objName_ru, soundUrl, objDescr, objViewUrl):
    self.soundsUrl = [] # перечень песен представителя фауны
    self.addSound(soundUrl)
    self.obj_spec = '' # латинское название вида представителя фауны. Состоит из нескольких слов
    self.setObjSpec(obj_spec) # сохранение вида представителя фауны
    self.objViewUrl = '' # URL фотографии представителя фауны
    self.setObjViewUrl(objViewUrl)
    self.objName_ru = '' # русское наименование представителя фауны
    self.setObjName_ru(objName_ru)
    self.objDescr = '' # Краткое описание представителя фауны
    self.setObjDescr(objDescr)

  
  # Задает видовое название представителя фауны в виде словаря для REST запроса
  def setObjSpec(self, obj_spec):
    self.obj_spec = obj_spec

  # задает русское наименование представителя фауны
  def setObjName_ru(self, objName_ru):
    self.objName_ru = objName_ru

  # Возвращает русское наименование представителя фауны
  def getObjName_ru(self):
    return self.objName_ru

  # задает URL фотографии представителя фауны
  def setObjViewUrl(self, objViewUrl):
    self.objViewUrl = objViewUrl

  # Возвращает URL фотографии представителя фауны
  def getObjViewUrl(self):
    return self.objViewUrl

  # задает URL краткого описания представителя фауны
  def setObjDescr(self, objDescr):
    self.objDescr = objDescr

  # Возвращает URL краткого описания представителя фауны
  def getObjDescr(self):
    return self.objDescr

  # добавляет очередную песню в список песен представителя фауны
  def addSound(self, sndUrl):
    self.soundsUrl.append(sndUrl)
  
  # возвращает песню представителя фауны выбранную случайным
  # образом из имеющихся для этого представителя фауны песен
  def getObjSongUrl(self):
    return self.soundsUrl[rd.randint(0, len(self.soundsUrl) - 1)]
class WikiREST_API:
  def __init__(self, wikiEndPoint):
    self.wikiUrl = wikiEndPoint # Конечная точка для wiki
    self.errGet = {} # словарь ошибок при выполнении запроса
    self.json = {} # полученные данные запроса в формате json

  # возвращает информацию в формате json для заданного словаря параметров params,
  # {'gen':..., 'sp'...}
  def getQueryData(self, params):
    resp = rq.get(self.wikiUrl + 'summary/' + params)
    if resp.status_code == 200:
      # Запрос выполнен успешно
      self.json = resp.json() # полученные json данные запроса
      return True
    else:
      self.errGet[params] = str(resp.status_code)
      return False
  
  # возвращает русское наименование выбранного объекта
  def getObjName_ru(self):
    return self.json['title']

  # возвращает ссылку на краткое описание объекта
  def getObjDescr(self):
    return self.json['extract']

  # возвращает ссылку на фотографию объекта
  def getObjPhotoUrl(self):
    return self.json['thumbnail']['source']


class ObjREST_API:
  def __init__(self, user, psw, endPoint, wikiEndPoint, qtyObj = 5):
    self.params = {"page": 1} # настраиваемые параметры запроса для сайта птиц
    self.user = user # Пользователь для сайта птиц
    self.psw = psw # Пароль пользователя для сайта птиц
    self.url = endPoint # Конечная точка для сайта птиц
    self.page = 1 # Стартовая страница запроса для сайта птиц
    self.numPages = 0 # количество страниц ответа на запрос для сайта птиц
    self.qtyObj = qtyObj # количество птиц для викторины
    self.obj = {} # данные о птицах в виде словаря: objName_ru : objInstance
    self.errGet = {} # словарь ошибок при выполнении запроса по страницам
    self.recObj = {} # словарь записей страницы птиц
    self.objNamesForRandom = [] # Список имен птиц для случайной выборки
    self.wikiObj = WikiREST_API(wikiEndPoint) # объект REST API для wiki
    self.wikiParams = '' # видовое описание объекта для запроса из wiki
  
  # возвращает № страницы, выбранной случайным образом из имеющихся в запросе
  def getRandomPage(self):
    return rd.randint(1, self.numPages)

  # возвращает выборку одной страницы ответа на запрос в json формате и количество страниц ответа на запрос
  def getDataObj(self, page = 1):
    # Получаем данные с сайта
    params = {}
    params["page"] = page
    resp = rq.get(self.url, params = self.params, auth=(self.user, self.psw), allow_redirects=True)
    if resp.status_code == 200:
      # Запрос выполнен успешно
      json = resp.json() # полученные json данные запроса заданной страницы
      self.numPages = json["numPages"] # Количество страниц в запросе
      self.recObj.clear() # предварительная очистка словаря записей птиц страницы
      self.recObj = json['recordings']
      return True
    else:
      self.errGet[page] = str(resp.status_code)
      return False

  # заполение списка объектов для викторины случайным образом
  def getRandomLstObj(self):
    global objPhotoUrl, objDescr
    if self.getDataObj(self.page) == False:
      # ошибка выполнения запроса
      return False
    # ответ получен
    while len(self.obj) <  self.qtyObj:
      # пока не занесено требуемое количество птиц
        # выбор случайной записи страницы
      record = self.recObj[rd.randint(0, len(self.recObj) - 1)]
      # Выборка описания объекта
      self.wikiParams = record['gen'] + ' ' + record['sp']
      # Выполнение запроса на wiki
      if self.wikiObj.getQueryData(self.wikiParams) == False:
        # ошибка выполнения запроса
        return False
      # Запрос выполнен успешно
      # создание экземпляра объекта
      objName_ru = self.wikiObj.getObjName_ru()
      if objName_ru not in self.obj:
        # Выбранная птица еще не занесена в список викторины
        self.obj[objName_ru] = Obj(self.wikiParams, objName_ru, record['file'], self.wikiObj.getObjDescr(), self.wikiObj.getObjPhotoUrl())
        objPhotoUrl = self.wikiObj.getObjPhotoUrl()
        objDescr = self.wikiObj.getObjDescr()
        self.objNamesForRandom.append(objName_ru)
      self.page = self.getRandomPage() # Выбор очередной страницы
      if self.getDataObj(self.page) == False:
        # ошибка выполнения запроса
        return False
    return True
    
  # возвращает список объектов, выбранных для викторины
  def getObjList(self):
    return self.obj

  # возвращает список ошибок выполнения запросов страниц
  # в формате {page: 'код ошибки: ....'}
  def getErrors(self):
    return self.errGet

  # Возвращает список имен объектов для случайной выборки
  def getObjNamesForRandom(self):
    return self.objNamesForRandom

objGetData = ObjREST_API(username, password, obj_url, wikiEndPoint)
if objGetData.getRandomLstObj() == True:
  LstObj = objGetData.getObjList()
  objNames = objGetData.getObjNamesForRandom()
  tstObj = LstObj[objNames[rd.randint(0, len(objNames) - 1)]]



In [15]:
LstObj[objNames[0]].getObjViewUrl()

'https://upload.wikimedia.org/wikipedia/commons/thumb/9/9b/Aix_galericulata_%28couple%29.jpg/320px-Aix_galericulata_%28couple%29.jpg'

In [11]:
LstObj[objNames[0]].getObjDescrUrl()

'Ря́бчик\xa0— птица из рода рябчиков, трибы тетеревиных, семейства фазановых отряда курообразных. Широко распространённый вид, обитающий практически повсеместно в лесной и таёжной зоне Евразии, от Западной Европы до Кореи. Рябчик\xa0— самый мелкий представитель тетеревиных. Масса тела даже самых крупных особей редко превышает 500 граммов. В лесу его сложно спутать с другими тетеревиными птицами, от которых он отличается не только небольшой величиной, но и достаточно узнаваемым окрасом. Несмотря на пёстрое, «рябое» оперение, уже с небольшого расстояния рябчик кажется однотонным, серо-рыжеватым. Половой диморфизм у рябчика выражен значительно слабее, чем у других тетеревиных\xa0— единственным заметным отличием самца от самки является чёрное оперение на горле. Кроме того, в отличие от других тетеревиных, рябчик\xa0— моногамная птица.'

In [ ]:
self.wikiObj.getObjPhotoUrl()

In [8]:
objPhotoUrl

'https://upload.wikimedia.org/wikipedia/commons/thumb/8/8e/Long-tailed-duck.jpg/320px-Long-tailed-duck.jpg'

In [9]:
objDescr

'Моря́нка, или саук, или аллейка, или савка, или аулейка\xa0— представитель семейства утиных, небольшая полярная нырковая утка. Легче всего определяется по очень длинному и тонкому хвосту самцов, который постоянно держится приподнятым над поверхностью воды. У морянок одна из наиболее сложных среди птиц система чередования нарядов, в результате чего как самцы, так и самки имеют различный окрас оперения зимой, летом, по окончании брачного сезона и в первый год жизни.'